# Example notebook

To run this notebook, you need:
1. An image file (named `test.tif`) with related segmentation masks (stored in `labels.tif`).
2. A file, in which the image and the masks have been merged so that they can be used in TrackMate together with the label detector.
3. The tracking result exported as a TrackMate XML, called `merged.ome.xml`
4. This example uses the FUCCI-SA sensor for HT1080 cells. Find these files in this repository and replace them if needed.

Then go through the notebook.
An example of a cropped video is shown.

In [ ]:
import json

import pandas as pd
from fucciphase import process_trackmate
from fucciphase.sensor import FUCCISASensor
from fucciphase.utils import postprocess_estimated_percentages
from fucciphase.phase import estimate_percentage_by_subsequence_alignment
from fucciphase.napari import add_trackmate_data_to_viewer, pandas_df_to_napari_tracks

from skimage.io import imread

import numpy as np

import napari
from aicsimageio import AICSImage
import vispy

from napari_animation import Animation
import os

In [ ]:
if not os.path.isdir("figures"):
    os.mkdir("figures")

In [ ]:
track_file = "merged.ome.xml"
cyan_channel_id = "MEAN_INTENSITY_CH2"
magenta_channel_id = "MEAN_INTENSITY_CH1"

In [ ]:
with open("fucciphase_processing/fuccisa_ht1080.json") as fp:
    sensor_properties = json.load(fp)
sensor = FUCCISASensor(**sensor_properties)
reference_track = pd.read_csv("fucciphase_processing/fuccisa_ht1080_reference.csv")

In [ ]:
track_df = process_trackmate(
    track_file,
    channels=[cyan_channel_id, magenta_channel_id],
    sensor=sensor,
    thresholds=[0.1, 0.1],
    use_moving_average=True,
    window_size=10,
    generate_unique_tracks=True
)

In [ ]:
reference_track.rename(columns={"cyan": cyan_channel_id, "magenta": magenta_channel_id}, inplace=True)
reference_track[cyan_channel_id + "_NORM"] = reference_track[cyan_channel_id]
reference_track[magenta_channel_id + "_NORM"] = reference_track[magenta_channel_id]


In [ ]:
estimate_percentage_by_subsequence_alignment(track_df, dt=0.25, 
                                             channels=[cyan_channel_id + "_NORM", magenta_channel_id + "_NORM"], 
                                             reference_data=reference_track,
                                             track_id_name="UNIQUE_TRACK_ID")
postprocess_estimated_percentages(track_df, "CELL_CYCLE_PERC_DTW", track_id_name="UNIQUE_TRACK_ID")

In [ ]:
labels = imread("labels.tif")

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.window.resize(1200, 900)
viewer.reset_view()

In [ ]:
image = AICSImage("test.tif")
scale = (image.physical_pixel_sizes.Y, image.physical_pixel_sizes.X)
labels = imread("manual_dapieq_labels.tif")
cyan = image.get_image_data("TYX", C=3)
magenta = image.get_image_data("TYX", C=0)

mag_1_perc = np.percentile(magenta, 1)
mag_99_9_perc = np.percentile(magenta, 99.9)
cyan_1_perc = np.percentile(cyan, 1)
cyan_99_9_perc = np.percentile(cyan, 99.9)

In [ ]:
viewer.scale_bar.unit = "um"
viewer.scale_bar.font_size = 0
viewer.scale_bar.ticks = False

In [ ]:
track_df["time"] = 0.25 * track_df["FRAME"]

# Napari settings

In [ ]:
old_time = -1
def update_slider(event):
    time = viewer.dims.current_step[0]
    global old_time
    if time != old_time:
        old_time = time
    else:
        return
    time = 15.0 * time
    viewer.text_overlay.text = f"{round(time)} min "

In [ ]:
viewer.text_overlay.color = "white"
viewer.text_overlay.blending = "translucent_no_depth"
viewer.text_overlay.position = "top_left"
viewer.text_overlay.font_size = 18
viewer.text_overlay.visible = True
viewer.dims.events.current_step.connect(update_slider)

In [ ]:
viewer.scale_bar.font_size = 0
viewer.scale_bar.unit = "um"
viewer.scale_bar.font_size = 0
viewer.scale_bar.visible = True

In [ ]:
viewer.dims.current_step = (0, 0, 0)

# Crop the video if needed

In [ ]:
x_limits = [50, 250]
y_limits = [100, 300]
crop_track_df = track_df[track_df["POSITION_X"] > x_limits[0]]
crop_track_df = crop_track_df[track_df["POSITION_X"] < x_limits[1]]
crop_track_df = crop_track_df[track_df["POSITION_Y"] > y_limits[0]]
crop_track_df = crop_track_df[track_df["POSITION_Y"] < y_limits[1]]
crop_track_df["POSITION_X"] -= x_limits[0]
crop_track_df["POSITION_Y"] -= y_limits[0]

x_limits_pixel = slice(int(x_limits[0] / scale[0]), int(x_limits[1] / scale[0])) 
y_limits_pixel = slice(int(y_limits[0] / scale[1]), int(y_limits[1] / scale[1]))

cyan_crop = cyan[..., y_limits_pixel, x_limits_pixel]
magenta_crop = magenta[..., y_limits_pixel, x_limits_pixel]
labels_crop = labels[..., y_limits_pixel, x_limits_pixel]

In [ ]:
add_trackmate_data_to_viewer(crop_track_df, viewer, scale=scale, 
                             image_data=[cyan_crop,
                                         magenta_crop], 
                             colormaps=["cyan", "magenta"],
                             labels=labels_crop, 
                             cycle_percentage_id="CELL_CYCLE_PERC_DTW",
                             label_id_name="MEDIAN_INTENSITY_CH3",
                             textkwargs={"size": 16})

In [ ]:
crop_track_df["CELL_CYCLE_PERC_NORM"] = crop_track_df["CELL_CYCLE_PERC_DTW"] / 100.0
colormap = vispy.color.colormap.MatplotlibColormap("cool")
pandas_df_to_napari_tracks(crop_track_df, viewer, unique_track_id_name="UNIQUE_TRACK_ID",
                           frame_id_name="FRAME", position_x_name="POSITION_X",
                           position_y_name="POSITION_Y", feature_name="CELL_CYCLE_PERC_NORM",
                           colormaps_dict={"CELL_CYCLE_PERC_NORM": colormap})

In [ ]:
viewer.layers[-1].tail_width = 6
viewer.layers[-1].blending = "translucent"
viewer.layers[1].contrast_limits = (cyan_1_perc, cyan_99_9_perc)
viewer.layers[2].contrast_limits = (mag_1_perc, mag_99_9_perc)

In [ ]:
frames_to_screenshot = [35, 50, 65, 80]
for frame in frames_to_screenshot:
    viewer.text_overlay.visible = True
    viewer.scale_bar.visible = True
    viewer.scale_bar.font_size = 16
    viewer.layers[-1].visible = True
    viewer.layers[0].visible = True
    
    viewer.dims.current_step = (frame, 0, 0)
    viewer.export_figure(f"frame_{frame}_with_points_and_tracks_and_labels.png")
    viewer.text_overlay.visible = False
    viewer.scale_bar.visible = False
    viewer.layers[-1].visible = False
    viewer.export_figure(f"frame_{frame}_with_tracks_and_labels.png")
    viewer.layers[0].visible = False
    viewer.export_figure(f"frame_{frame}_with_tracks.png")
    viewer.layers[-1].visible = True
    viewer.layers[0].visible = True


In [ ]:
animation = Animation(viewer)
viewer.text_overlay.visible = True
viewer.scale_bar.visible = True
viewer.scale_bar.font_size = 0
# start animation on first frame
viewer.dims.current_step = (0, 0, 0)
animation.capture_keyframe()
# last frame
viewer.dims.current_step = (labels.shape[0] - 1, 0, 0)
animation.capture_keyframe(steps=labels.shape[0] - 1)
animation.animate("movie_tails.mp4",
                   canvas_only=True, fps=4, quality=5, scale_factor=1.0)